<a href="https://colab.research.google.com/github/Maheen-Sabir/colab/blob/main/catvlion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install ultralytics scikit-learn pandas
from ultralytics import YOLO
model = YOLO("yolov8n.pt")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Y0DFeqxQjqDoIE5Pkwxu")
project = rf.workspace("learn-jt151").project("cat-ll4pq-mtmul")
version = project.version(3)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Cat--3 in yolov8:: 100%|██████████| 1720/1720 [00:00<00:00, 1992.14it/s]


In [ ]:
model.train(data='/content/Cat--3/data.yaml', epochs=10)

Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Cat--3/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0,

In [ ]:
from ultralytics import YOLO

# custom data trained model (lion, cat)
species_model = YOLO("/content/runs/detect/train/weights/best.pt")

# Generic COCO model for context objects (person, couch, bed, chair, etc.)
context_model = YOLO("/content/yolov8n.pt")

TARGETS_INDOOR = {"couch", "bed", "chair", "dining table", "tv", "laptop", "keyboard", "sink", "refrigerator"}

def detect_species(image_path):
    return species_model(image_path)

def detect_context(image_path):
    return context_model(image_path)

def get_label_map(results):
    return results[0].names  # id -> name

def box_area_rel(box_xywh, img_h, img_w):
    w, h = float(box_xywh[2].item()), float(box_xywh[3].item())
    return (w * h) / (img_h * img_w)

def aspect_ratio(box_xywh):
    w, h = float(box_xywh[2].item()), float(box_xywh[3].item())
    return w / max(h, 1e-9)

def any_indoor_object(ctx_results):
    if len(ctx_results[0].boxes) == 0:
        return False
    names = ctx_results[0].names
    for cls_id in ctx_results[0].boxes.cls:
        name = names[int(cls_id.item())].lower()
        if name in TARGETS_INDOOR:
            return True
    return False

def best_person_box(ctx_results):
    """Return the largest person box (xywh) or None."""
    r = ctx_results[0]
    if len(r.boxes) == 0:
        return None
    names = r.names
    person_ids = [k for k, v in names.items() if v.lower() == "person"]
    if not person_ids:
        return None
    pid = person_ids[0]

    idxs = [i for i, c in enumerate(r.boxes.cls) if int(c.item()) == pid]
    if not idxs:
        return None

    # choose largest area person
    img_h, img_w = r.orig_shape
    def area_i(i):
        b = r.boxes.xywh[i]
        return (float(b[2].item()) * float(b[3].item())) / (img_h * img_w)
    best_i = max(idxs, key=area_i)
    return r.boxes.xywh[best_i]

def rules_for_swap(pred_label, conf, box_xywh, img_h, img_w, ctx_results):
    reasons = []
    penalty = 0.0
    new_label = pred_label.lower()
    area = box_area_rel(box_xywh, img_h, img_w)
    ar = aspect_ratio(box_xywh)
    indoor = any_indoor_object(ctx_results)
    person_box = best_person_box(ctx_results)

    # --- Rule C: indoor scene ---
    if indoor and new_label == "lion" and conf < 0.90:
        new_label = "cat"
        reasons.append("Indoor context (furniture) => lion unlikely; relabel to cat")
        penalty += 0.15

    # --- Rule A: size plausibility ---
    if new_label == "lion" and area < 0.02 and conf < 0.80:
        new_label = "cat"
        reasons.append(f"Too small for lion (area={area:.3f}) and not high conf; relabel to cat")
        penalty += 0.20

    if new_label == "cat" and area > 0.18 and conf < 0.80:
        new_label = "lion"
        reasons.append(f"Too large for cat (area={area:.3f}) and not high conf; relabel to lion")
        penalty += 0.20

    # --- Rule C: compare with person scale ---
    if person_box is not None:
        animal_h = float(box_xywh[3].item())
        person_h = float(person_box[3].item())
        ratio = animal_h / max(person_h, 1e-9)

        if ratio < 0.25 and new_label == "lion" and conf < 0.92:
            new_label = "cat"
            reasons.append(f"Animal much smaller than person (h ratio={ratio:.2f}) => cat-like; relabel to cat")
            penalty += 0.15

        if ratio > 0.45 and new_label == "cat" and conf < 0.92:
            new_label = "lion"
            reasons.append(f"Animal fairly large vs person (h ratio={ratio:.2f}) => lion-like; relabel to lion")
            penalty += 0.10

    # --- Rule B: shape plausibility (soft) ---
    if ar > 1.6 and new_label == "cat" and conf < 0.85:
        penalty += 0.08
        reasons.append(f"Elongated box (w/h={ar:.2f}) => slightly lion-like; penalize cat")

    if ar < 1.1 and new_label == "lion" and conf < 0.85:
        penalty += 0.08
        reasons.append(f"Compact box (w/h={ar:.2f}) => slightly cat-like; penalize lion")

    return new_label, penalty, reasons

def final_decision(conf, penalty, accept_thr=0.55):
    final_score = max(0.0, conf - penalty)
    return ("ACCEPT" if final_score >= accept_thr else "REJECT"), final_score

# ---- Run example ----
image_path = "forest_neuro_symbolic_ai/data/images/lion.jpg"

sp = detect_species(image_path)
ctx = detect_context(image_path)

if len(sp[0].boxes) == 0:
    print("No species detected")
    raise SystemExit

# Choose highest-confidence species detection
i = int(sp[0].boxes.conf.argmax().item())
conf = float(sp[0].boxes.conf[i].item())
cls_id = int(sp[0].boxes.cls[i].item())
pred_label = sp[0].names[cls_id]

box = sp[0].boxes.xywh[i]
img_h, img_w = sp[0].orig_shape

new_label, penalty, reasons = rules_for_swap(pred_label, conf, box, img_h, img_w, ctx)
decision, final_score = final_decision(conf, penalty, accept_thr=0.55)

print("Raw:", pred_label, conf)
print("After rules:", new_label)
print("Decision:", decision)
print("Final score:", final_score)
print("Reasons:", reasons)
